# Dynamic Pricing for Urban Parking Lots

### Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import pathway as pw
from bokeh.plotting import figure, show, output_notebook # Import output_notebook

# Configure Bokeh to output to the notebook
output_notebook()

This is not the real Pathway package.
Visit https://pathway.com/developers/ to get Pathway.
Already tried that? Visit https://pathway.com/troubleshooting/ to get help.
Note: your platform is Windows-11-10.0.26100-SP0, your Python is CPython 3.13.1.


Loading BokehJS ...

### 📊 Loading & Exploring Data

In [2]:
# Load data
df = pd.read_csv("C:\\Users\\91739\\Downloads\\dataset.csv")

# Check first 5 rows
print(df.head())

# Basic stats
print("\nStatistics:")
print(df.describe())

   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime  
0        07:59:00  
1        08:25:00  
2        08:59:00  
3     

## 💰 Pricing Models
### (1) Baseline Linear Model


In [3]:
def baseline_linear_model(prev_price, occupancy, capacity, alpha=0.5):
    occupancy_rate = occupancy / capacity
    new_price = prev_price + (alpha * occupancy_rate)
    return max(new_price, 10)

In [4]:
print(baseline_linear_model(10, 50, 100))

10.25


### 📊 Visualizing the Baseline Linear Model

In [5]:
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource
import numpy as np

# Define a range of occupancy rates from 0 to 1 (0% to 100%)
occupancy_rates = np.linspace(0, 1, 100)

# Assume a previous price (e.g., 10) for demonstration
prev_price = 10
capacity = 100 # Assume a capacity to calculate occupancy

# Calculate the resulting price for each occupancy rate using the baseline model
# The baseline_linear_model expects occupancy and capacity, so we'll calculate occupancy from the rate
baseline_prices = [
    baseline_linear_model(prev_price, rate * capacity, capacity)
    for rate in occupancy_rates
]

# Create a ColumnDataSource
source = ColumnDataSource(data=dict(occupancy_rate=occupancy_rates * 100,  # Convert to percentage for plotting
                                     price=baseline_prices))

# Create a plot
p = figure(title="Baseline Linear Model: Price vs. Occupancy Rate",
           x_axis_label="Occupancy Rate (%)",
           y_axis_label="Price ($)",
           width=800, height=400)

# Add a line glyph
p.line(x='occupancy_rate', y='price', source=source, line_width=2)

show(p)

## (2) Demand-Based Model

In [6]:
def demand_based_model(base_price, occupancy, capacity, queue_length, traffic, is_special_day, vehicle_type):
    # Weights (tune these based on your data)
    weights = {
        'occupancy': 0.4,
        'queue': 0.3,
        'traffic': {'low': -0.2, 'medium': 0, 'high': 0.3, 'average': 0.1}, # Added 'average'
        'special_day': 0.5,
        'vehicle': {'car': 0, 'bike': -0.1, 'truck': 0.2, 'cycle': -0.05} # Added 'cycle'
    }

    demand = (
        weights['occupancy'] * (occupancy / capacity) +
        weights['queue'] * queue_length +
        weights['traffic'][traffic] +
        weights['special_day'] * is_special_day +
        weights['vehicle'][vehicle_type]
    )

    normalized_demand = max(0, min(1, demand))
    new_price = base_price * (1 + normalized_demand)
    return max(base_price * 0.5, min(new_price, base_price * 2))

In [7]:
test_price = demand_based_model(
    base_price=10,
    occupancy=80,
    capacity=100,
    queue_length=5,
    traffic="high",
    is_special_day=1,
    vehicle_type="truck"
)
print(test_price)

20


## ⚡ Real-Time Simulation

In [8]:
# Compute prices directly using pandas apply as a workaround for Pathway output issues
results_df = df.copy() # Create a copy to avoid modifying the original DataFrame

results_df['price'] = results_df.apply(
    lambda row: demand_based_model(
        base_price=10,
        occupancy=row['Occupancy'],
        capacity=row['Capacity'],
        queue_length=row['QueueLength'],
        traffic=row['TrafficConditionNearby'],
        is_special_day=row['IsSpecialDay'],
        vehicle_type=row['VehicleType']
    ),
    axis=1 # Apply the function row-wise
)


# Convert to Pandas DataFrame for visualization - Data is already in pandas
# pw.outputs.add_output("priced_data", priced_table)
# pw.run(loop=False) # Run the Pathway computation

# results_df = pw.outputs.get_output("priced_data").to_pandas()

print("\nFirst 5 priced records:")
print(results_df.head())


First 5 priced records:
   ID SystemCodeNumber  Capacity   Latitude  Longitude  Occupancy VehicleType  \
0   0      BHMBCCMKT01       577  26.144536  91.736172         61         car   
1   1      BHMBCCMKT01       577  26.144536  91.736172         64         car   
2   2      BHMBCCMKT01       577  26.144536  91.736172         80         car   
3   3      BHMBCCMKT01       577  26.144536  91.736172        107         car   
4   4      BHMBCCMKT01       577  26.144536  91.736172        150        bike   

  TrafficConditionNearby  QueueLength  IsSpecialDay LastUpdatedDate  \
0                    low            1             0      04-10-2016   
1                    low            1             0      04-10-2016   
2                    low            2             0      04-10-2016   
3                    low            2             0      04-10-2016   
4                    low            2             0      04-10-2016   

  LastUpdatedTime      price  
0        07:59:00  11.422877  

### 📊 Visualizing Price vs. Occupancy